In [22]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF


In [23]:
f4_inputs = np.load("initial_inputs.npy")
f4_outputs = np.load("initial_outputs.npy")
f4_inputs_2 = np.load("initial_inputs2.npy")
f4_outputs_2 = np.load("initial_outputs2.npy")

In [24]:
f4_inputs = np.concatenate((f4_inputs, f4_inputs_2)) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.533179, 0.320368, 0.682501, 0.312599]])))
f4_inputs = np.concatenate((f4_inputs, np.array([[0.444629, 0.320368, 0.808875, 0.908167]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.607765, 0.358771, 0.405825, 0.251007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.687501, 0.84587 , 0.833788, 0.143186]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.557765, 0.448771, 0.445825, 0.269007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.507765, 0.498771, 0.395825, 0.319007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.457765, 0.548771, 0.345825, 0.369007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.407765, 0.498771, 0.295825, 0.419007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.357765, 0.448771, 0.245825, 0.469007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.325901, 0.423101, 0.406001, 0.415501]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.287012, 0.439767, 0.456001, 0.432167]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.38256 , 0.363705, 0.389898, 0.390221]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.331802, 0.389754, 0.375876, 0.369452]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.331882, 0.379388, 0.371487, 0.422793]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.301994, 0.331229, 0.44261 , 0.408753]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.402732, 0.392983, 0.412051, 0.411811]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.394307, 0.324568, 0.266869, 0.332541]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.395203, 0.398125, 0.414027, 0.449168]]))) 

f4_outputs = np.concatenate((f4_outputs,f4_outputs_2))
f4_outputs = np.append(f4_outputs,-7.457831737943366)
f4_outputs = np.append(f4_outputs,-20.190789652235313)
f4_outputs = np.append(f4_outputs,-4.588426873216477)
f4_outputs = np.append(f4_outputs,-24.332647737297297)
f4_outputs = np.append(f4_outputs,-3.685845588546609)
f4_outputs = np.append(f4_outputs,-2.8122115680390354)
f4_outputs = np.append(f4_outputs,-2.590293835023562)
f4_outputs = np.append(f4_outputs,-2.0998084663837484)
f4_outputs = np.append(f4_outputs,-2.368538951646705)
f4_outputs = np.append(f4_outputs,0.13490612991991258)
f4_outputs = np.append(f4_outputs,-1.3532237087009196)
f4_outputs = np.append(f4_outputs,0.15448865125965527)
f4_outputs = np.append(f4_outputs,-0.02620181983396419)
f4_outputs = np.append(f4_outputs,0.19106899681514777)
f4_outputs = np.append(f4_outputs,-1.0065878357155316)
f4_outputs = np.append(f4_outputs,0.48394520320252754)
f4_outputs = np.append(f4_outputs,-1.70575402025506)
f4_outputs = np.append(f4_outputs,-0.1020328389196119)


In [25]:
variable_value = np.linspace(1, 10, 10)
n_dims = 4
beta =0.001# 1.1818181818181819
np.random.seed(42)
train_X = torch.tensor(f4_inputs)
train_Y = torch.tensor(f4_outputs).reshape(len(f4_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.9 #6.909090909090909
candidate_list = []
best_x_list = []

for i in range(len(variable_value)):
    #noise_level = variable_value[i]
   
    def function_8(x1,x2,x3,x4, noise_level=noise_level):
        return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
            np.sin(1 * x3+4*x4) * (1 - np.tanh(x3 ** 2) - np.tanh(x4)) + \
                   + np.random.randn() * noise_level
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    
    EI = qExpectedImprovement(model=gp , best_f=best_y)

    #UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=400, raw_samples=400,
    )


    x1,x2,x3,x4 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_8(x1,x2,x3,x4)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3,x4


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [26]:
best_x_list

[(0.523822009563446,
  0.07155434787273407,
  0.4387533366680145,
  0.8218817710876465),
 (0.523822009563446,
  0.07155434787273407,
  0.4387533366680145,
  0.8218817710876465),
 (0.4771196246147156,
  0.05013681948184967,
  0.45030006766319275,
  0.8646277189254761),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607),
 (0.4813847243785858,
  0.09431882947683334,
  0.4510539472103119,
  0.8816327452659607)]